# Wikimedia Discovery - Clickthrough and search analysis

The following analysis is a report for the [Wikimedia Discovery department](https://www.mediawiki.org/wiki/Wikimedia_Discovery). The goals of the project are:

1. "What is our daily overall clickthrough rate? How does it vary between the groups?
2. Which results do people tend to try first? How does it change day-to-day?
3. What is our daily overall zero results rate? How does it vary between the groups?
4. Let session length be approximately the time between the first event and the last event in a session. Choose a variable from the dataset and describe its relationship to session length. Visualize the relationship.
5. Summarize your findings in an executive summary."

This is a demo project intended for the Wikimedia data scientist job application. See [link](https://github.com/wikimedia-research/Discovery-Hiring-Analyst-2016) for more details.

In [28]:
# imports
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

In [83]:
# read data as gzip file
df = pd.read_csv('events_log.csv.gz', compression='gzip', header=0, sep=',', quotechar='"')

In [84]:
df['timestamp']=df['timestamp'].astype(int) #convert to int to slice
df['timestamp']=pd.to_datetime(df['timestamp'],format='%Y%m%d%H%M%S', errors='ignore') #convert to pandas datetime

From the original documentation:

| Column          | Value   | Description                                                                       |
|:----------------|:--------|:----------------------------------------------------------------------------------|
| uuid            | string  | Universally unique identifier (UUID) for backend event handling.                  |
| timestamp       | integer | The date and time (UTC) of the event, formatted as YYYYMMDDhhmmss.                |
| session_id      | string  | A unique ID identifying individual sessions.                                      |
| group           | string  | A label ("a" or "b").                                     |
| action          | string  | Identifies in which the event was created. See below.                             |
| checkin         | integer | How many seconds the page has been open for.                                      |
| page_id         | string  | A unique identifier for correlating page visits and check-ins.                    |
| n_results       | integer | Number of hits returned to the user. Only shown for searchResultPage events.      |
| result_position | integer | The position of the visited page's link on the search engine results page (SERP). |

In [85]:
# get acclimated...
print(df.head(3))

                               uuid           timestamp        session_id  \
0  00000736167c507e8ec225bd9e71f9e5 2016-03-01 10:38:42  78245c2c3fba013a   
1  00000c69fe345268935463abbfa5d5b3 2016-03-07 00:52:26  c559c3be98dca8a4   
2  00003bfdab715ee59077a3670331b787 2016-03-02 14:53:05  760bf89817ce4b08   

  group            action  checkin           page_id  n_results  \
0     b  searchResultPage      NaN  cbeb66d1bc1f1bc2        5.0   
1     a  searchResultPage      NaN  eb658e8722aad674       10.0   
2     a           checkin     30.0  f99a9fc1f7fdd21e        NaN   

   result_position  
0              NaN  
1              NaN  
2              NaN  


In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400165 entries, 0 to 400164
Data columns (total 9 columns):
uuid               400165 non-null object
timestamp          400165 non-null datetime64[ns]
session_id         400165 non-null object
group              400165 non-null object
action             400165 non-null object
checkin            223824 non-null float64
page_id            400165 non-null object
n_results          136234 non-null float64
result_position    230482 non-null float64
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 27.5+ MB


## 1. Clickthrough rate

From wikipedia, "Click-through rate (CTR) is the ratio of users who click on a specific link to the number of total users who view a page, email, or advertisement."

In this case, we want to compare the number of search links clicked to the number of search pages viewed.

$CTR = \frac{clicked}{total pages}$

We also want to do this for groups 'a' and 'b'...so let's subset the original dataframe.

In [87]:
group_a=df.loc[df['group'] == 'a']
group_b=df.loc[df['group'] == 'b']

len_grp_a=len(group_a)
len_grp_b=len(group_b)

We have {{len_grp_a}} UUIDs in group A, and {{len_grp_b}} in group B.

In [88]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
from contents import key
plotly.tools.set_credentials_file(username='wnowak10', api_key=key)

x = ['a','b']
y = [len_grp_a, len_grp_b]
df = pd.DataFrame({'x': x, 'y': y})
df.head()

data = [
    go.Bar(
        x=df['x'], # assign x as the dataframe column 'x'
        y=df['y']
    )
]
#py.iplot(data)

layout=go.Layout(title="Size of user groups (a and b)", xaxis={'title':'Group'}, yaxis={'title':'# UUIDs'})
figure=go.Figure(data=data,layout=layout)
py.iplot(figure, filename='pyguide_1')

We have {{number_sessions}} unique sessions. Let's learn more about each session before we get around to finding the CTR. 

In [7]:
len(df.index)

400165

In [8]:
#df.groupby('session_id').size() # how long is each session (this pandas code is super sweet)

In [9]:
df['action'].value_counts()

checkin             223824
searchResultPage    136234
visitPage            40107
Name: action, dtype: int64

In [10]:
clicked_links=df['action'].value_counts()[2]
ctr=clicked_links/number_sessions
print(ctr)

0.589566061034


In [11]:
matplotlib.pyplot.hist(sizes,bins=10,range=(0,50))
plt.show()

NameError: name 'sizes' is not defined

Here, we can see that most of the sessions are contain just 0-20 UUIDs...while there are some people who do stay logged in for longer. If we assume that the time stamps are in a format of YYYYMMDDHHMMSS, then we can find  103842

The maximum session contains {{df.groupby('session_id').size().max()}} UUIDs.

The maximum session contains {{df.groupby('session_id').size().max()}} UUIDs.

In [ ]:
#longest_session=df[df.session_id==df.session_id.value_counts().index.tolist()[15003]]
#longest_session.sort_values(by='timestamp')